# Comprendre les mécanismes d'attention

In [27]:
import tensorflow as tf
import math


In [28]:

# Initialisation
sentence = "Quel temps fait-il"
embedding_layer = tf.keras.layers.Embedding(5000, 256)  
tokenized_sentence = [15, 120, 260, 45]  # Tokenisation arbitraire pour l'exemple
embedded_sentence = embedding_layer(tf.convert_to_tensor([tokenized_sentence]))
print("Dimension de l'embedding de la phrase:", embedded_sentence.shape)


Dimension de l'embedding de la phrase: (1, 4, 256)


In [29]:

# Encodeur
## Calcul des Query, Key, Value
Q_enc = tf.keras.layers.Dense(256)(embedded_sentence)
K_enc = tf.keras.layers.Dense(256)(embedded_sentence)
V_enc = tf.keras.layers.Dense(256)(embedded_sentence)
print("\nEncodeur:")
print("Dimension de Q:", Q_enc.shape)
print("Dimension de K:", K_enc.shape)
print("Dimension de V:", V_enc.shape)



Encodeur:
Dimension de Q: (1, 4, 256)
Dimension de K: (1, 4, 256)
Dimension de V: (1, 4, 256)


In [30]:

## Attention
QK_enc = tf.matmul(Q_enc, K_enc, transpose_b=True)
QK_normalized_enc = QK_enc / tf.math.sqrt(tf.cast(256, tf.float32))
softmax_enc = tf.nn.softmax(QK_normalized_enc)
attention_enc = tf.matmul(softmax_enc, V_enc)
print("Dimension de l'attention:", attention_enc.shape)


Dimension de l'attention: (1, 4, 256)


In [31]:

# Décodeur (pour prédire le prochain mot)
## Utilisez l'état du dernier mot comme Query
Q_dec = Q_enc[:, -1, :]
print("\nDécodeur:")
print("Dimension de Q (décodeur):", Q_dec.shape)




Décodeur:
Dimension de Q (décodeur): (1, 256)


In [32]:

## Attention
QK_dec = tf.matmul(tf.expand_dims(Q_dec, 1), K_enc, transpose_b=True)
QK_normalized_dec = QK_dec / tf.math.sqrt(tf.cast(256, tf.float32))
softmax_dec = tf.nn.softmax(QK_normalized_dec)
attention_dec = tf.matmul(softmax_dec, V_enc)
print("Dimension de l'attention (décodeur):", attention_dec.shape)


Dimension de l'attention (décodeur): (1, 1, 256)


In [33]:

# Sortie: prédiction du prochain mot
output = tf.keras.layers.Dense(5000, activation='softmax')(attention_dec)  
print("\nDimension de la sortie:", output.shape)



Dimension de la sortie: (1, 1, 5000)


In [34]:

# Prédire le mot avec l'indice le plus élevé comme mot suivant
predicted_next_word_index = tf.argmax(output, axis=-1)
print("Index du mot prédit:", predicted_next_word_index.numpy())

Index du mot prédit: [[1558]]


# Multihead Attention

In [35]:
import tensorflow as tf
import math


In [41]:
def split_heads(x, num_heads):
    """Divise les dernières dimensions de x en (num_heads, depth)."""
    batch_size = tf.shape(x)[0]
    d_model = x.shape[-1]
    depth = d_model // num_heads

    reshaped_x = tf.reshape(x, (batch_size, -1, num_heads, depth))
    return tf.transpose(reshaped_x, perm=[0, 2, 1, 3])

def multi_head_attention(Q, K, V, num_heads):
    """Implémentation de la multi-head attention."""
    d_model = Q.shape[-1]
    depth = d_model // num_heads

    # Divise en plusieurs têtes
    Q = split_heads(Q, num_heads)
    K = split_heads(K, num_heads)
    V = split_heads(V, num_heads)

    # Calcul de l'attention pour chaque tête
    QK = tf.matmul(Q, K, transpose_b=True)
    QK_normalized = QK / tf.math.sqrt(tf.cast(depth, tf.float32))
    softmax_weights = tf.nn.softmax(QK_normalized, axis=-1)
    attention = tf.matmul(softmax_weights, V)

    # Concatène les têtes et applique une transformation linéaire
    attention_concatenated = tf.transpose(attention, perm=[0, 2, 1, 3])
    concatenated = tf.reshape(attention_concatenated, (tf.shape(attention)[0], -1, d_model))
    return tf.keras.layers.Dense(d_model)(concatenated)

In [42]:
# Initialisation
sentence = "Le ciel est"
embedding_layer = tf.keras.layers.Embedding(5000, 256)  
tokenized_sentence = [10, 100, 150]
embedded_sentence = embedding_layer(tf.convert_to_tensor([tokenized_sentence]))
print("Dimension de l'embedding de la phrase:", embedded_sentence.shape)

Dimension de l'embedding de la phrase: (1, 3, 256)


In [44]:

# Encodeur avec multi-head attention
## Calcul des Query, Key, Value
Q_enc = tf.keras.layers.Dense(256)(embedded_sentence)
K_enc = tf.keras.layers.Dense(256)(embedded_sentence)
V_enc = tf.keras.layers.Dense(256)(embedded_sentence)

## Multi-head attention
attention_enc = multi_head_attention(Q_enc, K_enc, V_enc, 8)
print("\nEncodeur:")
print("Dimension après multi-head attention:", attention_enc.shape)




Encodeur:
Dimension après multi-head attention: (1, 3, 256)


In [39]:

# Décodeur avec multi-head attention
## Utiliser l'état du dernier mot comme Query
Q_dec = Q_enc[:, -1, :]
K_dec = K_enc
V_dec = V_enc

## Multi-head attention
attention_dec = multi_head_attention(tf.expand_dims(Q_dec, 1), K_dec, V_dec, 8)
print("\nDécodeur:")
print("Dimension de l'attention multi-tête (décodeur):", attention_dec.shape)



Décodeur:
Dimension de l'attention multi-tête (décodeur): (1, 1, 256)


In [40]:

# Sortie: prédiction du prochain mot
output = tf.keras.layers.Dense(5000, activation='softmax')(attention_dec)  
print("\nDimension de la sortie:", output.shape)

# Prédire le mot avec l'indice le plus élevé comme mot suivant
predicted_next_word_index = tf.argmax(output, axis=-1)
print("Index du mot prédit:", predicted_next_word_index.numpy())



Dimension de la sortie: (1, 1, 5000)
Index du mot prédit: [[3390]]
